In [73]:
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm
from zipfile import ZipFile
import io
import OSGridConverter
import folium

In [17]:
PLACE_NAMES = "../big-data/opname_csv_gb.zip"
COLUMN_NAMES = "DOC/OS_Open_Names_Header.csv"

In [33]:
data = []
with ZipFile(PLACE_NAMES) as z:
    
    with z.open(COLUMN_NAMES) as a:
        headers = pd.read_csv(io.BytesIO(a.read()), header=None)
        headers = headers.T[0].tolist()
    
    for f in tqdm(z.namelist()):
        if not f.endswith(".csv") or not f.startswith("DATA/"):
            continue
        with z.open(f) as a:
            df = pd.read_csv(io.BytesIO(a.read()), header=None, names=headers)
            df = df[df['NAME1'].fillna("").str.contains("Point") | df['NAME2'].fillna("").str.contains("Point")]
            data.append(df)
data = pd.concat(data)

c:\users\drkan\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\users\drkan\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (4,16,17,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [55]:
g = OSGridConverter.OSGridReference(data.iloc[0]['GEOMETRY_X'], data.iloc[0]['GEOMETRY_Y']).toLatLong()
g.latitude
g.longitude

-1.0670102467336122

In [58]:
latlongs = data.apply(
    lambda x: OSGridConverter.OSGridReference(x['GEOMETRY_X'], x['GEOMETRY_Y']).toLatLong(),
    axis=1
)

In [60]:
data.loc[:, "LATITUDE"] = latlongs.apply(lambda x: x.latitude)
data.loc[:, "LONGITUDE"] = latlongs.apply(lambda x: x.longitude)

In [61]:
data[["NAME1", "LATITUDE", "LONGITUDE"]]

,NAME1,LATITUDE,LONGITUDE
54,Point of Hellia,60.728852,-1.067010
55,Cockle Point,60.685879,-0.905768
56,Point of Whack,60.731580,-1.070214
121,Cross Point,60.719651,-0.955058
247,Point of Coppister,60.723260,-0.955711
...,...,...,...
5716,Back Sand Point,51.297011,1.357279
6,Hope Point,51.370606,1.452680
1333,Flagstaff Point,50.749896,0.180620
1352,Brass Point,50.751658,0.173932


In [65]:
gdf = gpd.GeoDataFrame(
    data,
    geometry=gpd.points_from_xy(data.LONGITUDE, data.LATITUDE)
)

In [71]:
gdf[["geometry", "NAME1"]].to_file("point_points.geojson", driver='GeoJSON')

In [90]:
m = folium.Map(tiles=None)
fg = folium.map.FeatureGroup("Points")
for index, row in gdf.iterrows():
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=0.5,
        tooltip=row['NAME1'],
        color='#3186cc',
        fill=False,
        fill_color='#3186cc'
    ).add_to(fg)
fg.add_to(m)
m.fit_bounds(fg.get_bounds())
m

In [91]:
m.save("01-point-points.html")